In [1]:
import os
import sys
import math
import copy
import time
import numpy
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.extern import six
from spectral_cube import SpectralCube

In [2]:
import aplpy
import astropy
from astropy.io import fits
from astropy.wcs import WCS
import warnings;warnings.filterwarnings('ignore')
from tqdm import tqdm

from astropy.units import deg
from astropy.units import km
from astropy.units import s

In [3]:
font = 'Helvetica'
plt.rcParams['font.size'] = 7
plt.rcParams['font.family'] = font
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = font
plt.rcParams['mathtext.it'] = font + ':italic'
plt.rcParams['mathtext.bf'] = font + ':bold'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

In [4]:
def func_make_mom012_3x1(data_dir_, data_fits_, v_unit_, int_start_v_, int_end_v_, save_dir_, cut_mask_, levels_=[0], cut_=0.0,
                     color_min0=0.0, color_max0=180.0, color_min1=-60.0, color_max1=-30.0, color_min2=0.0, color_max2=10.0, i=0):
    
    t0 = time.time()
    
    hdulist = fits.open(data_dir_ + data_fits_)
    hdr = hdulist[0].header
    data0 = hdulist[0].data
    w = WCS(data_dir_ + data_fits_)

    if v_unit_ == 'm/s':
        int_start_ch = int(w.wcs_world2pix(0.0, 0.0, int_start_v_*1000.0, 0)[2])
        int_end_ch = int(w.wcs_world2pix(0.0, 0.0, int_end_v_*1000.0, 0)[2])
        print('(start_ch, end_ch) = ({0}, {1})'.format(int_start_ch, int_end_ch))
        CDELT3 = hdr['CDELT3']/1000.0#km/s
    elif v_unit_ == 'km/s':
        #start_l_ch, start_b_ch, int_start_ch = w.wcs_world2pix(0.0, 0.0, int_start_v_, 0)
        #end_l_ch, end_b_ch, int_end_ch = w.wcs_world2pix(0.0, 0.0, int_end_v_, 0)
        CDELT3 = hdr['CDELT3']#km/s


#     save_name = data_fits_[:-5]+"%s..mom012.%.1f_%.1f.integcut_%.1f.mask_%.1sigma"%(mode, int_start_v_, int_end_v_, cut_, sigma_)#save name

    data_new = data0[int_start_ch:int_end_ch+1, :, :]
    hdr_new = copy.deepcopy(hdr)
    hdr_new['CRPIX3'] = hdr_new['CRPIX3'] - int_start_ch
    hdr_new["NAXIS3"] = int_end_ch - int_start_ch + 1
    hdr_new["CTYPE3"] = "VELO-LSR"
    
    hdr_new["CTYPE1"] = "GLON-CAR"
    hdr_new["CTYPE2"] = "GLAT-CAR"
    hdr_new["CTYPE3"] = "VELO-LSR"
    hdr_new['BLANK'] = -1
    #del hdr_new["CTYPE3"]
    try:
        del hdr_new["VOBS"]
    except:
        pass
    w_new = WCS(hdr_new)    
    #print w_new
    #hdu_new = fits.PrimaryHDU(data_new, hdr_new)
#     if cut_!=0.0:   data_new[(data_new<cut_) & (data_new>-cut_)] = np.nan

    S = SpectralCube(data_new, w_new)
#     print w_new
    mom0_map = S.moment0().value/1000.0
    data_new_mask = copy.deepcopy(data_new)
    data_new_mask[:,(mom0_map<cut_mask_)] = np.nan
    S_mask = SpectralCube(data_new_mask, w_new)
    mom1_map = S_mask.moment1().value/1000.0
    #if cut_!=0.0:   data_new_mask[(data_new_mask<cut_) & (data_new_mask>-cut_)] = np.nan
    if cut_!=0.0:   data_new_mask[(data_new_mask<cut_)] = np.nan
    S_mask_clip = SpectralCube(data_new_mask, w_new)
    mom2_map = np.sqrt(S_mask_clip.moment2().value)/1000.0*2.0*math.sqrt(2.0*math.log(2))
    #mom2_map = S_mask.linewidth_fwhm().value
    
    t1 = time.time()
    print('Caluculation Spectral Cube finished. [time : {0:.2f} [sec.]]'.format(t1 - t0))

    try:
        del hdr["CRVAL3"]
    except:
        pass
    try:
        del hdr["CRPIX3"]
    except:
        pass
    try:
        del hdr["CRVAL3"]
    except:
        pass
    try:
        del hdr["CDELT3"]
    except:
        pass
    try:
        del hdr["CUNIT3"]
    except:
        pass
    try:
        del hdr["CTYPE3"]
    except:
        pass
    try:
        del hdr["CROTA3"]
    except:
        pass
    try:
        del hdr["NAXIS3"]
    except:
        pass
    try:
        hdr["NAXIS"] = 2
    except:
        pass
    try:
        del hdr["WCSAXES"]
    except:
        pass
    hdr["NAXIS"] = 2

    #return fits.PrimaryHDU(mom0_map, hdr), fits.PrimaryHDU(mom1_map, hdr), fits.PrimaryHDU(mom2_map, hdr)
    hdu0, hdu1, hdu2 = fits.PrimaryHDU(mom0_map, hdr), fits.PrimaryHDU(mom1_map, hdr), fits.PrimaryHDU(mom2_map, hdr)
    
    hdulist = fits.HDUList([hdu1])
    
    # ---
    hdulist.writeto('/Users/r.yamada/science/fixedheader12CO_1.fits', overwrite=True)
    # ---

    fig = pl.figure(figsize=(24, 12))
    
    ### mom0

    f = aplpy.FITSFigure(hdu0, slices=[0],convention='wells', subplot=[0.05*1.0+0.8/3.0*0.0, 0.05, 0.8/3.0, 0.9], figure=fig)
    f.show_colorscale(vmin=color_min0, vmax=color_max0, stretch='linear', cmap=cmap_mom0, aspect="equal")
    f.add_colorbar()
    f.colorbar.show()
    f.colorbar.set_width(0.2)
    f.colorbar.set_font(size=10, family='helvetica')
    f.colorbar.set_axis_label_text("Moment 0 (K km s$^{-1}$)")
    f.colorbar.set_axis_label_font(size=14, family='helvetica')
    f.colorbar.set_location("top")
    f.recenter(mapcenter_l,mapcenter_b,width=mapsize_l,height=mapsize_b)
    #f.show_rgb(save_png_name)
#     f.ticks.set_xspacing(0.3)  # degrees
#     f.ticks.set_yspacing(0.3)  # degrees
    f.ticks.set_length(5, minor_factor=0.5)  # points
    f.ticks.set_color('k')
    f.ticks.set_linewidth(1.5)  # points
    f.ticks.set_minor_frequency(5)
    f.tick_labels.set_xformat('dd.d')
    f.tick_labels.set_yformat('dd.d')
    f.tick_labels.set_font(size=15, family='helvetica')
    f.axis_labels.set_font(size=15, family='helvetica')
    f.add_scalebar(math.asin(5. / 2000.) / math.pi * 180.0)
    f.scalebar.set_corner('bottom right')
    f.scalebar.set_label('5 pc')
    f.scalebar.set_font_family('helvetica')
    f.scalebar.set_font_size(15)
    f.scalebar.set_color("k")
    f.scalebar.set_linewidth(3)
#     f.show_markers(col_ostar_l, col_ostar_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')        
#     f.show_markers(stars_W_l, stars_W_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars_O_l, stars_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars_B_l, stars_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_A_l, stars2_A_b, marker="x", s=60, linewidth=0.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_B_l, stars2_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.set_title(title, size=20, family='helvetica', y=1.02)
    f.show_contour(hdu0, levels=levels_, convention='wells', colors="k", linewidths=0.6, slices=[0], linestyles="-")
#     f.save(os.path.join(save_dir, object_name, save_name0)+'.pdf')


    t2 = time.time()
    print('Moment 0 map plot finished. [time : {0:.2f} [sec.]]'.format(t2 - t1))
    
    ### mom1
    f = aplpy.FITSFigure(hdu1, slices=[0],convention='wells', subplot=[0.05*2.0+0.8/3.0*1.0, 0.05, 0.8/3.0, 0.9], figure=fig)
    f.show_colorscale(vmin=color_min1, vmax=color_max1, stretch='linear', cmap=cmap_mom1, aspect="equal")
    f.add_colorbar()
    f.colorbar.show()
    f.colorbar.set_width(0.2)
    f.colorbar.set_font(size=10, family='helvetica')
    f.colorbar.set_axis_label_text("Moment 1 (km s$^{-1}$)")
    f.colorbar.set_axis_label_font(size=14, family='helvetica')
    f.colorbar.set_location("top")
    f.recenter(mapcenter_l,mapcenter_b,width=mapsize_l,height=mapsize_b)
    #f.show_rgb(save_png_name)
#     f.ticks.set_xspacing(0.4)  # degrees
#     f.ticks.set_yspacing(0.4)  # degrees
    f.ticks.set_length(5, minor_factor=0.5)  # points
    f.ticks.set_color('k')
    f.ticks.set_linewidth(1.5)  # points
    f.ticks.set_minor_frequency(5)
    f.tick_labels.set_xformat('dd.d')
    f.tick_labels.set_yformat('dd.d')
    f.tick_labels.set_font(size=15, family='helvetica')
    f.axis_labels.set_font(size=15, family='helvetica')
    f.add_scalebar(math.asin(5. / 2000.) / math.pi * 180.0)
    f.scalebar.set_corner('bottom right')
    f.scalebar.set_label('5 pc')
    f.scalebar.set_font_family('helvetica')
    f.scalebar.set_font_size(15)
    f.scalebar.set_color("k")
    f.scalebar.set_linewidth(3)
#     f.show_markers(col_ostar_l, col_ostar_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')    
#     f.show_markers(stars_W_l, stars_W_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars_O_l, stars_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars_B_l, stars_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_A_l, stars2_A_b, marker="x", s=60, linewidth=0.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_B_l, stars2_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.set_title(title, size=20, family='helvetica', y=1.02)
#     f.show_contour(hdu0, levels=levels_, convention='wells', colors="k", linewidths=0.6, slices=[0], linestyles="-")
    f.show_contour(hdu0, levels=[levels_[0]], convention='wells', colors="k", linewidths=0.6, slices=[0], linestyles="-")
#     f.save(os.path.join(save_dir, object_name, save_name1)+'.pdf')


    t3 = time.time()
#     print('Moment 1 map plot finished. [time : {0:.2f} [sec.]]'.format(t3 - t2))
    
    ### mom2
    f = aplpy.FITSFigure(hdu2, slices=[0],convention='wells', subplot=[0.05*3.0+0.8/3.0*2.0, 0.05, 0.8/3.0, 0.9], figure=fig)
    f.show_colorscale(vmin=color_min2, vmax=color_max2, stretch='linear', cmap=cmap_mom2, aspect="equal")
    f.add_colorbar()
    f.colorbar.show()
    f.colorbar.set_width(0.2)
    f.colorbar.set_font(size=10, family='helvetica')
    f.colorbar.set_axis_label_text("HPBW (km s$^{-1}$)")
    f.colorbar.set_axis_label_font(size=14, family='helvetica')
    f.colorbar.set_location("top")
    f.recenter(mapcenter_l,mapcenter_b,width=mapsize_l,height=mapsize_b)
    #f.show_rgb(save_png_name)
#     f.ticks.set_xspacing(0.4)  # degrees
#     f.ticks.set_yspacing(0.4)  # degrees
    f.ticks.set_length(5, minor_factor=0.5)  # points
    f.ticks.set_color('k')
    f.ticks.set_linewidth(1.5)  # points
    f.ticks.set_minor_frequency(5)
    f.tick_labels.set_xformat('dd.d')
    f.tick_labels.set_yformat('dd.d')
    f.tick_labels.set_font(size=15, family='helvetica')
    f.axis_labels.set_font(size=15, family='helvetica')
    f.add_scalebar(math.asin(5. / 2000.) / math.pi * 180.0)
    f.scalebar.set_corner('bottom right')
    f.scalebar.set_label('5 pc')
    f.scalebar.set_font_family('helvetica')
    f.scalebar.set_font_size(15)
    f.scalebar.set_color("k")
    f.scalebar.set_linewidth(3)
#     f.show_markers(col_ostar_l, col_ostar_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')    
#     f.show_markers(stars_W_l, stars_W_b, marker="*", s=100, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars_O_l, stars_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars_B_l, stars_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_A_l, stars2_A_b, marker="x", s=60, linewidth=0.5, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_O_l, stars2_O_b, marker="x", s=60, linewidth=1.0, edgecolor='k', facecolor='k')
#     f.show_markers(stars2_B_l, stars2_B_b, marker="^", s=60, linewidth=1.0, edgecolor='k', facecolor='none')
#     f.set_title(title, size=20, family='helvetica', y=1.02)
#     f.show_contour(hdu0, levels=levels_, convention='wells', colors="k", linewidths=0.6, slices=[0], linestyles="-")
    f.show_contour(hdu0, levels=[levels_[0]], convention='wells', colors="k", linewidths=0.6, slices=[0], linestyles="-")
#     f.save(os.path.join(save_dir, object_name, save_name)+'.pdf')

    t4 = time.time()
    print('Moment 2 map plot finished. [time : {0:.2f} [sec.]]'.format(t4 - t3))

    # fits の書き出し
    hdulist = fits.HDUList([hdu1])
    hdulist.writeto('/Users/r.yamada/science/fixedheader12CO.fits',overwrite=True)
    
    # 画像保存
    #f.save('/Users/sakasaikeisuke/Desktop/moment_map_{}sigma.png'.format(i), dpi=200)
    #print('/// finished.')

In [5]:
rms = 0.11218886
num_sigma = 0

#hdu = fits.open('../fits/12CO_2to1_alittle_header.fits')[0]
hdu = fits.open('/Users/r.yamada/science/fixedheader12CO.fits')[0]
data = hdu.data
header = hdu.header

# --- calc sigma.
v_interval = hdu.header['CDELT3'] / 1000 # km / s
vmin = 8. # km / s
vmax = 15. # km / s
ch_num = (vmax - vmin) / v_interval
sigma = numpy.sqrt(ch_num + 1) * rms # ch number * rms
# contour_levels = numpy.array([1,8,16,24,32,40]) * 10 * sigma
contour_levels = numpy.array([0.5,1,2,4,6,8,10,12,14,16,18,20])*0.86*10.0
sigma_cube = rms * v_interval
cut_mask = rms * numpy.sqrt(ch_num + 1) * v_interval * 10

mode = '12CO21'
cmap_mom0 = 'CMRmap_r'
cmap_mom1 = 'jet'
cmap_mom2 = 'jet'
mapcenter_l, mapcenter_b, mapsize_l, mapsize_b = 133.6, 0.8, 1.6, 1.6
#txt_dict = numpy.loadtxt('../data/annotation_otype.txt', dtype={'names':('1', '2'),'formats':('f8', 'f8')}, skiprows=0, delimiter=',')
#col_ostar_l = txt_dict['1']
#col_ostar_b = txt_dict['2']
#save_dir = '.'

In [6]:
func_make_mom012_3x1("../science/", "fixedheader12CO.fits", 'm/s', int_start_v_=vmin, int_end_v_=vmax, save_dir_='.',
                     levels_ = contour_levels, cut_mask_=cut_mask, cut_ = sigma_cube * 5,
                    color_min0=0.0, color_max0=180.0, color_min1=-55., color_max1=-35., color_min2=0., color_max2=10.)

(start_ch, end_ch) = (359, 382)
Caluculation Spectral Cube finished. [time : 0.19 [sec.]]


IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

<Figure size 1728x864 with 1 Axes>